<a href="https://colab.research.google.com/github/cku7808/DL-for-everyone2/blob/main/ML_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Softmax Classification 

앞서 로지스틱 회귀에서 사용한 시그모이드 함수는 입력된 데이터에 대해서 0과 1사이의 값을 출력하여 2개의 선택지 중 1개를 고르는 이진 분류 문제를 해결했다.  
소프트맥스 회귀는 3개 이상의 선택지 중 1개를 고르는 다중 클래스 분류 문제를 해결한다.

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)

data 준비

In [7]:
#feature가 4개인 8개의 데이터
x_data = [[1,2,1,1],
          [2,1,3,2],
          [3,1,3,4],
          [4,1,5,5],
          [1,7,5,5],
          [1,2,5,6],
          [1,6,6,6],
          [1,7,7,7]]
          
#class가 3개인 정답(라벨)데이터
y_data = [[0,0,1],
          [0,0,1],
          [0,0,1],
          [0,1,0],
          [0,1,0],
          [0,1,0],
          [1,0,0],
          [1,0,0]]

#np.asarray : 원본 변경 시 복사본도 변경
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [8]:
nb_classes = 3 #class 개수
print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [9]:
#가중치와 바이어스 결정
#tf.random.normal : 주어진 형태의 난수를 갖는 텐서 반환
W = tf.Variable(tf.random_normal([4,nb_classes]), name="weight")
b = tf.Variable(tf.random_normal([nb_classes]), name="bias")
variables = [W,b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


softmax 함수 사용

In [10]:
#softmax 구현
def hypothesis(X):
  return tf.nn.softmax(tf.matmul(X, W)+b)

print(hypothesis(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162291e-03 9.78441179e-01]
 [3.92597765e-02 1.70347411e-02 9.43705440e-01]
 [3.80385250e-01 1.67723149e-01 4.51891541e-01]
 [3.23390484e-01 5.90759404e-02 6.17533624e-01]
 [3.62997366e-06 6.20727221e-08 9.99996245e-01]
 [2.62520202e-02 1.07279625e-02 9.63019967e-01]
 [1.56525093e-05 4.21802724e-07 9.99983847e-01]
 [2.94076904e-06 3.81133241e-08 9.99996960e-01]], shape=(8, 3), dtype=float32)


In [11]:
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


loss function 정의(크로스 엔트로피)

$$cost = - \sum\limits_{i}y_i log(\hat{y}_i)$$

In [12]:
#손실함수 : 크로스엔트로피
def cost_fn(X,Y):
  logits = hypothesis(X)
  cost = -tf.reduce_sum(Y*tf.log(logits), axis=1)
  cost_mean = tf.reduce_mean(cost)

  return cost_mean

print(cost_fn(x_data,y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


Gradient descent 계산

In [13]:
def grad_fn(X,Y):
  with tf.GradientTape() as tape:
    loss = cost_fn(X,Y)
    grads = tape.gradient(loss, variables)

    return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914607, -0.6509784 ,  0.5818323 ],
       [-1.5221257 , -1.214863  ,  2.7369885 ],
       [-1.2473829 , -1.7611003 ,  3.0084827 ],
       [-1.2014607 , -1.8659232 ,  3.0673835 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.15212914, -0.342192  ,  0.49432108], dtype=float32)>]


학습 진행

In [15]:
def fit(X,Y,epochs=2000, verbose=100):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

  for i in range(epochs):
    grads = grad_fn(X,Y)
    optimizer.apply_gradients(zip(grads, variables))
    if(i==0) | ((i+1)%verbose==0):
      print("Loss at epoch %d: %f" %(i+1, cost_fn(X,Y).numpy()))

fit(x_data, y_data)

Loss at epoch 1: 0.159427
Loss at epoch 100: 0.154003
Loss at epoch 200: 0.148872
Loss at epoch 300: 0.144060
Loss at epoch 400: 0.139539
Loss at epoch 500: 0.135282
Loss at epoch 600: 0.131269
Loss at epoch 700: 0.127480
Loss at epoch 800: 0.123896
Loss at epoch 900: 0.120502
Loss at epoch 1000: 0.117282
Loss at epoch 1100: 0.114226
Loss at epoch 1200: 0.111319
Loss at epoch 1300: 0.108553
Loss at epoch 1400: 0.105917
Loss at epoch 1500: 0.103402
Loss at epoch 1600: 0.101001
Loss at epoch 1700: 0.098706
Loss at epoch 1800: 0.096510
Loss at epoch 1900: 0.094407
Loss at epoch 2000: 0.092391


prediction check

In [17]:
a = hypothesis(x_data)
print(a)
print(tf.argmax(a,1))

tf.Tensor(
[[2.9436805e-08 1.5021235e-04 9.9984980e-01]
 [4.3665289e-04 4.6358626e-02 9.5320481e-01]
 [9.5460018e-10 9.6164070e-02 9.0383589e-01]
 [2.2774371e-07 9.1306245e-01 8.6937323e-02]
 [1.6050829e-01 8.3273548e-01 6.7562237e-03]
 [8.4970415e-02 9.1502690e-01 2.6765201e-06]
 [8.2284725e-01 1.7714940e-01 3.2996732e-06]
 [9.6834594e-01 3.1654030e-02 1.4558012e-08]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
